In [1]:
import numpy as np
import pandas as pd

%matplotlib inline

In [ ]:
"""data_epochs_1 = X_all_subjects['1'][:,:,:2560]
data_epochs_1 = data_epochs_1.reshape(2560,15,160)
print(data_epochs_1.shape)
"""

In [ ]:
"""X = compute_feature_matrix(data_epochs_1, 512)
Y = y_all_subjects['1']
X.shape
"""

In [15]:
X = pd.read_csv("./data/2-processing/bnci-001-2014-1-features.csv")

Y = pd.read_csv('./data/labels_004_1.csv', header=None)

In [16]:
X.shape

(461, 23)

In [17]:
Y.shape

(1, 5184)

In [18]:
labels = Y[:461]

In [19]:
labels.shape

(1, 5184)

In [24]:
labels = labels.to_numpy()[0]

In [26]:
labels.shape

(5184,)

In [ ]:
import numpy as np
import numpy.fft as fft
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

import pandas as pd

In [ ]:
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, labels, test_size=0.1, random_state=seed)

# Test options and evaluation metric

scoring = 'accuracy'

# Spot Check Algorithms
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB(var_smoothing=0.001)))
models.append(('SVM', SVC()))
# evaluate each model in turn
results = []
names = []

for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state=seed)
    cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)



In [ ]:
lda = models[1][1]
lda.fit(X,labels)

In [ ]:
from joblib import dump, load
dump(lda, './models/modelLR.joblib') 

In [ ]:
clf = load('./models/modelLR.joblib') 

In [ ]:
clf.predict(X)

In [ ]:
labels.shape

In [ ]:
labels.to_numpy()